In [1]:
!pip install pytrends

  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=9f043e9c513637606b5b088c42f046921bbb1aa4455c7316412a30639a503c14
  Stored in directory: /Users/francoismizrahi/Library/Caches/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


In [2]:
import pandas as pd

In [8]:
df = pd.read_csv("../data/google_trends_gamestop.csv")

In [25]:
df = df.iloc[1:]

In [26]:
df.columns = ['values']

In [27]:
df["values"].unique()

array(['1', '0', '<\xa01', '2', '3', '4', '5', '6', '7', '9', '8', '100',
       '69', '29', '15', '12', '14'], dtype=object)

In [28]:
len(df)

220